# I) Importation de toutes les bibliothèques importantes

In [1]:
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import random

# Deep Learning
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,  SeparableConv2D, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2

from keras.preprocessing.image import ImageDataGenerator

C:\Users\Utilisateur\Anaconda3\envs\deeplearning\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# II) Création des variables utiles

In [2]:
labels = ["1" , "2" , "3" , "4"]
electrodes = ["C3" , "C4" , "Cz"]

racine_scalo = "/content/drive/MyDrive/Projet_E4_Interface_Cerveau_Machine/Bases de données EEG/Bases de données/bdd_scalo"
racine_spectro = "/content/drive/MyDrive/Projet_E4_Interface_Cerveau_Machine/Bases de données EEG/Bases de données/bdd_spectro"

# III) Ecriture des différentes fonctions utiles

In [15]:
#fonction qui marche, import les données

def import_data(nom_electrode, racine) :
  data = []
  for j in range(len(labels)) :
    #on trouve les chemins vers les données scalo et spectro
    path_img_scalo = racine + "/" + labels[j] + "/" + nom_electrode + "/"
    #on récupère la liste des chemins pour les images scalo et spectro
    list_scalo = glob.glob(path_img_scalo + '*png' ) 
    #on enregistre chaque image à sa place 
    for l in range(len(list_scalo)) :
        #scalo
        img_scalo = cv2.imread(list_scalo[l])
        #spectro
        data.append([img_scalo, labels[j]])
  return data

In [16]:
def separation_data(data):
  X = []
  y = []

  for i in range(len(data)):
    X.append(data[i][0])
    y.append(int(data[i][1]))

  random.seed(4)
  random.shuffle(X)
  random.shuffle(y)

  n = int(len(X) * 0.8)
  n_ = int(len(X)-5)
  X_train = X[:n]
  X_test = X[n:n_]
  X_val = X[n_:]

  y_train = y[:n]
  y_test = y[n:n_]
  y_val = y[n_:]

  def normalize_image(img_set):
    img_set = np.array(img_set)/255
    return img_set
  X_train = normalize_image(X_train)
  X_val = normalize_image(X_val)
  X_test = normalize_image(X_test)

  X_train = np.array(X_train)
  X_test = np.array(X_test)
  X_val = np.array(X_val)

  y_train = np.array(y_train)
  y_test = np.array(y_test)
  y_val = np.array(y_val)

  return X_train, X_test, X_val, y_train, y_test, y_val

In [17]:
def creation_model(input_shape) :
  model_1 = Sequential()

  model_1.add(Conv2D(8, (3,3), input_shape = input_shape[1:]))
  model_1.add(Activation("relu"))
  model_1.add(MaxPool2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.2))

  model_1.add(Conv2D(16, (3,3), input_shape = input_shape[1:]))
  model_1.add(Activation("relu"))
  model_1.add(MaxPool2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.2))

  model_1.add(Conv2D(32, (3,3)))
  model_1.add(Activation("relu"))
  model_1.add(MaxPool2D(pool_size=(2, 2)))
  model_1.add(Dropout(0.2))

  model_1.add(Flatten())
  model_1.add(Dense(32))
  model_1.add(Activation("relu"))
  model_1.add(Dropout(0.5))
  model_1.add(Dense(1))
  model_1.add(Activation("softmax"))

  return model_1

In [18]:
def fit_model(model, epochs, batch_size, steps, X_train, X_val, y_train, y_val):
  import gc
  gc.collect()

  datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True) #on ne rescale pas car on l'a déja fait avant (normalisation des données : /255)

  datagen.fit(X_train)
  # fits the model on batches with real-time data augmentation:
  history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch= steps, 
          epochs=epochs,
          validation_data = (X_val, y_val))
  return history

# Test de bouclage sur les electrodes pour voir l'efficacité du réseau en fonction du réseau

**Problème de RAM, la session plante...**

In [ ]:
historys = []

for i in electrodes :
  data = import_data(i)
  X_train, X_test, X_val, y_train, y_test, y_val = separation_data(data)
  model = creation_model(np.shape(X_train))
  model.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  epochs = 10
  batch_size = int(len(X_train)/ 6)
  steps = int(len(X_train) / 32)
  history = fit_model(model, epochs, batch_size, steps, X_train, X_val, y_train, y_val)
  historys.append(history)
  print(history)

Epoch 1/10
10/10 [==============================] - 49s 2s/step - loss: -150.7686 - accuracy: 0.1890 - val_loss: -5465.4258 - val_accuracy: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: -12086.6394 - accuracy: 0.2849 - val_loss: -143885.7188 - val_accuracy: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: -171602.1818 - accuracy: 0.2185 - val_loss: -1305892.0000 - val_accuracy: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 17s 2s/step - loss: -1249542.9943 - accuracy: 0.2635 - val_loss: -6917821.0000 - val_accuracy: 0.0000e+00
Epoch 5/10
10/10 [==============================] - 17s 2s/step - loss: -5919661.6364 - accuracy: 0.2506 - val_loss: -26662548.0000 - val_accuracy: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 17s 2s/step - loss: -21133936.6364 - accuracy: 0.2522 - val_loss: -83306992.0000 - val_accuracy: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 17s 2s/s

# Entrainement d'un réseau sur l'électrode C3 pour les spectro et les scalogrammes

In [8]:
def importation_entrainement(electrode, racine) :
  data = import_data(electrode, racine)
  X_train, X_test, X_val, y_train, y_test, y_val = separation_data(data)
  model = creation_model(np.shape(X_train))
  model.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  epochs = 5
  batch_size = int(len(X_train)/ 6)
  steps = int(len(X_train) / 32)
  history = fit_model(model, epochs, batch_size, steps, X_train, X_val, y_train, y_val)
  return history

**Electrode C3 : test sur les scalogrammes**

In [9]:
test_1 = importation_entrainement(electrodes[0], racine_scalo)

Epoch 1/5
10/10 [==============================] - 49s 2s/step - loss: -188.8263 - accuracy: 0.2458 - val_loss: -1340.3792 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 17s 2s/step - loss: -3157.9348 - accuracy: 0.2326 - val_loss: -13440.3906 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 17s 2s/step - loss: -20107.2346 - accuracy: 0.2422 - val_loss: -72810.1641 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 17s 2s/step - loss: -87037.9975 - accuracy: 0.2332 - val_loss: -296877.1875 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 16s 2s/step - loss: -304259.1634 - accuracy: 0.2420 - val_loss: -1039263.8125 - val_accuracy: 0.0000e+00


In [10]:
data_scalo = import_data(electrodes[0], racine_scalo)
X_train1, X_test1, X_val1, y_train1, y_test1, y_val1 = separation_data(data_scalo)

In [11]:
test_1.model.predict(X_test1)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]]

**Electrode C3 : test sur les spectrogrammes**

In [9]:
test_2 = importation_entrainement(electrodes[0], racine_spectro)

Epoch 1/5
10/10 [==============================] - 49s 2s/step - loss: -521.5488 - accuracy: 0.2447 - val_loss: -3687.4995 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 17s 2s/step - loss: -7260.1995 - accuracy: 0.2537 - val_loss: -31190.3652 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 16s 2s/step - loss: -47505.8771 - accuracy: 0.2397 - val_loss: -178041.7188 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 16s 2s/step - loss: -203287.1420 - accuracy: 0.2468 - val_loss: -776237.3750 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 17s 2s/step - loss: -830794.7045 - accuracy: 0.2277 - val_loss: -2685469.7500 - val_accuracy: 0.0000e+00


In [10]:
data_spectro = import_data(electrodes[0], racine_spectro)
X_train2, X_test2, X_val2, y_train2, y_test2, y_val2 = separation_data(data_spectro)

In [11]:
test_2.model.predict(X_test2)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]]

In [18]:
from keras.models import load_model
test_1.model.save('C3_scal.h5')
test_2.model.save('C3_spec.h5')
#model = load_model('my_model.h5')

**On visualise les prédictions des deux modèles**

In [20]:
model1 = keras.models.load_model('C3_scal.h5')
model2 = keras.models.load_model('C3_spec.h5')

In [ ]:
data_scalo = import_data(electrodes[0], racine_scalo)
X_train1, X_test1, X_val1, y_train1, y_test1, y_val1 = separation_data(data_scalo)

data_spectro = import_data(electrodes[0], racine_spectro)
X_train2, X_test2, X_val2, y_train2, y_test2, y_val2 = separation_data(data_spectro)

In [1]:
y_pred1 = model1.predict(X_test1)
y_pred2 = model1.predict(X_test2)

NameError: ignored

In [ ]:
import seaborn as sns

sns.countplot(y_test1)
sns.countplot(y_pred1)

# Test de concaténation de deux modèles

In [8]:
model1 = keras.models.load_model('C3_scal.h5')
model2 = keras.models.load_model('C3_spec.h5')

In [13]:
y_test1

array([1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 3, 1, 2, 4, 3, 4, 3, 3, 1, 1, 1, 2,
       1, 2, 1, 1, 4, 3, 4, 4, 3, 2, 1, 2, 4, 2, 4, 4, 1, 3, 1, 1, 4, 2,
       3, 2, 4, 3, 4, 3, 3, 1, 2, 1, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 4, 3,
       2, 4, 2, 2, 2, 2, 4, 3, 2, 4, 3])

In [9]:
data_scalo = import_data(electrodes[0], racine_scalo)
X_train1, X_test1, X_val1, y_train1, y_test1, y_val1 = separation_data(data_scalo)

data_spectro = import_data(electrodes[0], racine_spectro)
X_train2, X_test2, X_val2, y_train2, y_test2, y_val2 = separation_data(data_spectro)

from keras.layers import Input, Concatenate, Dense
from keras.models import Model

model1_in = Input(shape=np.shape(X_train1))
model1_out = model1(model1_in)
#model1_final = Model(model1_in, model1_out)

model2_in = Input(shape=np.shape(X_train2))
model2_out = model2(model2_in)
#model2_final = Model(model3_in, model3_out)

concatenated = Concatenate()([model1_out, model2_out])
out = Dense(1, activation='softmax', name='output_layer')(concatenated)

merged_model = Model([model1_in, model2_in], out)
merged_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: ignored

In [ ]:
  merged_model.fit([x1, x2], y=y, batch_size=384, epochs=5, verbose=1, validation_split=0.1)
  
  epochs = 5
  batch_size = int(len(X_train)/ 6)
  steps = int(len(X_train) / 32)
  history = fit_model(model, epochs, batch_size, steps, X_train, X_val, y_train, y_val)